# <center> Домашнее задание
<center> EDA-4. СТАТИСТИЧЕСКИЕ ТЕСТЫ



**Задача:**
>
Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [1]:
import pandas as pd
import numpy as np

In [2]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

Для проведения статистических тестов требуется объединение массивов в DataFrame. Массивы являются разной длинны, по-этому добавим к массиву petersburg дополнительное значение, которое равно среднему и не будет конфликтовать с верностью наших расчетов и выводов по выборке.

In [3]:
petersburg.append(sum(petersburg)/len(petersburg))
d = {'petersburg':petersburg, 'magadan':magadan}
data = pd.DataFrame(d)
data.head(8)

,petersburg,magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


Выполним тесты на нормальность распределения каждого массива по методу Шапиро-Уилка. Тест Д'Агостино, провести не можем так, как число наблюдений меньше 20. В зависимости от результата выберем тест на сравнение выборок. Этот тест даст ответ, есть или нет значимая разница в размерах мидий. 

In [4]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05

In [5]:
from scipy.stats import shapiro
_, p = shapiro(data['petersburg'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.123
Данные распределены нормально


In [6]:
from scipy.stats import shapiro
_, p = shapiro(data['magadan'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.036
Данные не распределены нормально (мы отвергаем H0)


Данные распределены нормально для Санкт-Петербурга, но для Магадана не нормально, следовательно можем выбрать корреляцию Спирмана.

In [49]:
from scipy.stats import spearmanr
corr, p = spearmanr(data['petersburg'], data['magadan'])
display(corr)

0.1666666666666667

Результат вычисления показал, что присутствует прямая очень слабая корреляция между размерами мидий в разных городах. Проверим, есть ли значимая разница в размере мидий в разных городах.

### Какова разница в среднем размере мидии в зависимости от города-производителя?

In [48]:
difference = data['petersburg'].mean() - data['magadan'].mean()
print('Разница в среднем размере мидий вычисленная в ручную равна %.5f' % difference)
print(round(difference / data['petersburg'].mean() * 100, 2),'% от среднего размера в Санкт-Петербурге.')


Разница в среднем размере мидий вычисленная в ручную равна 0.02543
24.58 % от среднего размера в Санкт-Петербурге.


Данные распределены не нормально, следовательно мы используем непараметрические тесты. 
Зависимая переменная (размер мидии) является количественной. Группы происходят из разных совокупностей. 
Проверка гипотезы с помощью статистического теста Уилкоксона.

In [45]:
from scipy.stats import wilcoxon

_, p = wilcoxon(data['petersburg'], data['magadan'])

H0 = 'Нет значимой разницы в размере мидии в разных городах.'
Ha = 'Есть значимая разница в размере мидии в разных городах.'
alpha = 0.05

print('\n' + "*** Результаты теста ***")

if p > alpha:
        print(f"{p} > {alpha}. {H0}") 
else:
    print(f"{p} <= {alpha}. {Ha}")


*** Результаты теста ***
0.0234375 <= 0.05. Есть значимая разница в размере мидии в разных городах.


После выполнения вычислений, p-value меньше уровня значимости. Мы можем отвергнуть нулевую гипотезу - это значить, что есть значимая разница в размере мидии в разных городах.
